In [25]:
import numpy as np
import pickle
from tqdm.auto import tqdm

In [31]:
def choose_most_freq_words(bag_of_words):
    freq = bag_of_words.sum(axis=0)  # count frequency every word
    sort_freq_index = np.argsort(freq)
    top_words_index = sort_freq_index[-int(len(freq) / 10):]  # 10% most freq words
    return top_words_index

In [28]:
def count_range_sentences(bag_of_words, top_words_index):
    sent_range_list = []
    for i, sent in enumerate(bag_of_words):  # take word-index string
        use_words_index = np.nonzero(sent)  
        words_count = len(use_words_index)
        top_words_count = len(np.intersect1d(top_words_index, use_words_index))

        sent_range = (top_words_count ** 2) / words_count
        sent_range_list.append(sent_range)
    return sent_range_list

In [27]:
def choose_sent_for_summ(sent_range_list, y_sent_count):
    sort_sent_range = np.argsort(sent_range_list)
    # 10% most important sentences in right order
#     sent_for_summ_index = np.sort(sort_sent_range[-int(len(sent_range_list) * 0.1):])

    # Y_sent_count most important sentences in right order
    sent_for_summ_index = np.sort(sort_sent_range[-y_sent_count:]) 
    return sent_for_summ_index

###Main

In [32]:
with open('X_tfidf_texts_test.pickle', 'rb') as f:
    X_tfidf_texts_test = pickle.load(f) # sparce matrix
with open('count_sent_in_text_Y_test.pickle', 'rb') as f:
    count_sent_in_text_Y_test = pickle.load(f)

In [33]:
all_sent_for_summ_index = []

for i, tfidf_text in tqdm(enumerate(X_tfidf_texts_test)):
    tfidf_text = tfidf_text.toarray()
    
    top_words_index = choose_most_freq_words(tfidf_text)
    sent_range_list = count_range_sentences(tfidf_text, top_words_index)
    sent_for_summ_index = choose_sent_for_summ(sent_range_list, count_sent_in_text_Y_test[i])
    all_sent_for_summ_index.append(sent_for_summ_index)
    
    if i%500==0:
        print(sent_for_summ_index)

0it [00:00, ?it/s]

[13 17]
[ 6 11 15 26]
[ 1 10 23]
[10 29 30]
[4 5 6]
[ 0  4  9 35]
[13 16 26]
[ 7 11 14]
[ 7 23 31 33]
[ 1 11 19]
[ 2 21]
[7]


In [30]:
with open('luhn_index_sent_for_summ_X_test.pickle', 'wb') as f:
    pickle.dump(np.array(all_sent_for_summ_index), f)

<ipython-input-30-ccd5d94aea61>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pickle.dump(np.array(all_sent_for_summ_index), f)
